## Scrapping data de Facundo Pellistri

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://fbref.com/en/players/ac7e7b1c/Facundo-Pellistri"
page = requests.get(url)
soup = BeautifulSoup(page.content)



In [10]:

id_elements_to_get = ["all_stats_standard", "all_stats_shooting","all_stats_passing", "all_stats_passing_types", "all_stats_gca", 
                      "all_stats_defense", "all_stats_possession", "all_stats_playing_time"]

In [11]:
def get_stats(stats_table):
    stats = {}
    for i, data in enumerate(stats_table.find_all("tr", id="stats")):
        year = data.find("th").text
        team = data.find("td", {"data-stat": "team"}).text
        stats[f"{year}-{team}"] = {}
        for cell in data.find_all("td"):       
            stats[f"{year}-{team}"][cell.get("data-stat")] = cell.text  
    return stats

In [12]:
def build_dataset(soup, tags):
    data = {}
    for tag in tags:
        stats_div = soup.find("div", id=tag)
        stats_table = stats_div.find("tbody")
        if tag not in data:
            data[tag] = {}
        data[tag] = get_stats(stats_table)
    return data

In [13]:
dataset = build_dataset(soup, id_elements_to_get)

In [16]:
all_stats_standard = pd.DataFrame.from_dict(dataset["all_stats_standard"], orient='index')
all_stats_shooting = pd.DataFrame.from_dict(dataset["all_stats_shooting"], orient='index')
all_stats_passing = pd.DataFrame.from_dict(dataset["all_stats_passing"], orient='index')
all_stats_passing_types = pd.DataFrame.from_dict(dataset["all_stats_passing_types"], orient='index')
all_stats_gca = pd.DataFrame.from_dict(dataset["all_stats_gca"], orient='index')
all_stats_defense = pd.DataFrame.from_dict(dataset["all_stats_defense"], orient='index')
all_stats_possession = pd.DataFrame.from_dict(dataset["all_stats_possession"], orient='index')
all_stats_playing_time = pd.DataFrame.from_dict(dataset["all_stats_playing_time"], orient='index')


## Scrapping data de equipos de Europa

#### Primero necesitamos ir por cada una de las cinco ligas grandes de Europa y obtener la url de cada equipo

In [20]:
url_leagues = {
    "premier_league":"https://fbref.com/es/comps/9/Estadisticas-de-Premier-League",
    "la_liga":"https://fbref.com/es/comps/12/Estadisticas-de-La-Liga",
    "ligue_1":"https://fbref.com/es/comps/13/Estadisticas-de-Ligue-1",
    "serie_a":"https://fbref.com/es/comps/11/Estadisticas-de-Serie-A",
    "bundesliga":"https://fbref.com/es/comps/20/Estadisticas-de-Bundesliga"
}

In [70]:
def get_teams_url(league_url):
    page = requests.get(league_url)
    soup = BeautifulSoup(page.content)
    url_fixed_path = "https://fbref.com/"
    positions_table = soup.find("table", {"class": "stats_table"})
    team_cells = positions_table.find_all("td", {"data-stat": "team"})

    teams_url = {}
    for cell in team_cells:
        team_ref = cell.find("a")
        teams_url[team_ref.text] = f"{url_fixed_path}{team_ref['href']}"
    return teams_url
    

In [71]:
url_teams = {}
for league, url in url_leagues.items():
    url_teams[league] = get_teams_url(url)
            
    

In [72]:
url_teams

{'premier_league': {'Liverpool': 'https://fbref.com//es/equipos/822bd0ba/Estadisticas-de-Liverpool',
  'Manchester City': 'https://fbref.com//es/equipos/b8fd03ef/Estadisticas-de-Manchester-City',
  'Arsenal': 'https://fbref.com//es/equipos/18bb7c10/Estadisticas-de-Arsenal',
  'Aston Villa': 'https://fbref.com//es/equipos/8602292d/Estadisticas-de-Aston-Villa',
  'Tottenham': 'https://fbref.com//es/equipos/361ca564/Estadisticas-de-Tottenham-Hotspur',
  'West Ham': 'https://fbref.com//es/equipos/7c21e445/Estadisticas-de-West-Ham-United',
  'Manchester Utd': 'https://fbref.com//es/equipos/19538871/Estadisticas-de-Manchester-United',
  'Brighton': 'https://fbref.com//es/equipos/d07537b9/Estadisticas-de-Brighton-and-Hove-Albion',
  'Chelsea': 'https://fbref.com//es/equipos/cff3d9bb/Estadisticas-de-Chelsea',
  'Newcastle Utd': 'https://fbref.com//es/equipos/b2b47a98/Estadisticas-de-Newcastle-United',
  'Wolves': 'https://fbref.com//es/equipos/8cec06e1/Estadisticas-de-Wolverhampton-Wanderers',